chs_env\Scripts\activate

In [5]:
# Importing all relevant and necessary modules
import numpy as np 
import pandas as pd 
import os
import chess
import math
import matplotlib.pyplot as plt
import ollama


In [6]:
def test_prompt(prompts,board):
    result = np.zeros(len(prompts))
    for i in range(len(prompts)):
        prompt = prompts[i]
        move = prompt_move(prompt,board)
        if move in  gen_legal_moves(board):
            result[i] = 1
    return result

def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

def gen_prompt2(board,moves, n=10):
    return f"You are playing chess and the board state is {board}. Here you are given a list of legal moves {moves}. You have to chose the best move to make next from the list of legal moves and provide the move exactly as they are given to you. What is your next move given in UCI notation?"
    

In [7]:
df = pd.read_csv(r"chessData.csv")